In [1]:
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium import webdriver
from time import sleep
import pandas as pd
import re

In [5]:
df = pd.read_csv("links.csv")
df

,youtube_link,caption,download_link,success,downloaded
0,https://youtu.be/EYr7tJCf8C4,NaN,NaN,NaN,NaN
1,https://youtu.be/5_rSxy7uk-0,NaN,NaN,NaN,NaN
2,https://youtu.be/rn_WmM45cGc,NaN,NaN,NaN,NaN
3,https://youtu.be/xH1BOnD0tOE,NaN,NaN,NaN,NaN
4,https://youtu.be/tErALPjnCIE,NaN,NaN,NaN,NaN
...,...,...,...,...,...
111,https://youtu.be/UHmsyHnKYFg,NaN,NaN,NaN,NaN
112,https://youtu.be/c3GSThHHcpw,NaN,NaN,NaN,NaN
113,https://youtu.be/sIWtjDZv3aA,NaN,NaN,NaN,NaN
114,https://youtu.be/owm5Vf1TM7k,NaN,NaN,NaN,NaN


In [2]:
def sanitize_filename(caption):
    caption = re.sub(r'Duration:\s*\d{1,2}:\d{2}\s*minutes$', '', caption, flags=re.MULTILINE)
    filename = re.sub(r'[^\w\s-]', '', caption)
    filename = re.sub(r'[\s\n]+', '_', filename)
    filename = filename.strip('_')
    return filename[:255]

In [6]:
def save_download_video(youtube_link):
    try:
        url = 'https://en.y2mate.is/v96/'

        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

        driver.get(url)

        driver.maximize_window()

        a = driver.find_elements(By.ID, "txtUrl")[0].send_keys(youtube_link)

        b = driver.find_elements(By.ID, "btnSubmit")[0].click()

        sleep(3)

        trs = driver.find_elements(By.TAG_NAME, 'tr')

        for i in trs:
            if "360p" in i.text:
                tr = i
                break

        tr.find_element(By.TAG_NAME, 'button').click()

        sleep(3)

        while True:

            try:
                download_btn = driver.find_element(By.ID, "btnOk")
                break

            except:
                sleep(3)
                continue

        x = download_btn.find_element(By.TAG_NAME, "button").find_element(By.TAG_NAME, "a")

        sleep(3)

        download_link = x.get_attribute("href")

        caption = driver.find_element(By.TAG_NAME, "b").text

        caption = sanitize_filename(caption)

        driver.quit()

        return download_link, caption
    
    except Exception as e:
        print(e)
        return None, None

In [7]:
save_download_video("https://youtu.be/EYr7tJCf8C4")

('https://srv1.dlmate.cc/dl?hash=wz81MdIXdMmBaln5TmBgKk2Dt8HhsSmrPMeQ9bFs9eq7KC%2B88G6rVF97X%2Fo64%2BA4EKrBIZBBUdrDTSQxXXV0XTaF%2F0UTiVsUzCGn3ZBZIVYPWZhHAaweCcRb6Oq6QV3igCovJxk8XrNXHNG8O%2FebxESSZxF4ogd8a5m5027nVdnW2Z2K4IdsdyYzH%2FECKnvIWRuRYlOJYLpNyvGm9hwoqGsjvlIlq99mVZvJ8ZtO%2BY8Xhho%2Fg2Yh6HW9rmz36aIrfJpV275ycTdgEBPSYr%2BhG4uAn2dxI4jinlWmwtRqLhV7Pw7Jibn8cVCu9Z7CtDJMfQ27tDVtqpAmIhKFJXtRmqB8o1O0HegpE9dM55nRtAtUP5qyO%2FrzZSBatobz31XlSWcYRRI9Qao0ajt%2BFK3uL7H9LdVeLfJzfwBYhpf8OemxQR%2BHJuZ6XHQYnM8WzKOHI2hsOjoJagOpVwOUG6LKplGkGtfimrbXmBWEqgsQuOdXzY1Sl9n409FxrB3Bmc5UNSeqrcAN5S%2BRd90Gppzpkr5RYOSRVi310iySdtebuO0vXoqwjybD05mwaQfCYF1%2F',
 'تئوری_آشوب_به_زبان_ساده_جلسه_۱_چارچوب_فیزیک_نظری_قسمت_۱_-_NATPHYS')

In [ ]:
for index, row in df.iterrows():

    download_link, caption = save_download_video(row['youtube_link'])

    df.at[index, 'download_link'] = download_link
    df.at[index, 'caption'] = caption
    df.at[index, 'success'] = True

    df.to_csv("links.csv", index=False)